In [2]:
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [3]:
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

raw_wiki= requests.get(wiki_url).text
soup = BeautifulSoup(raw_wiki,'html5lib')

In [4]:
table=soup.find('table')

In [5]:
Postcode= []
Borough= []
Neighbourhood = []

In [6]:
x=0 #counter
for table_cell in table.find_all('td'):
    
    x = x+1
    if (x==1):
        postal_code = table_cell.text
    elif (x==2):
        borough = table_cell.text
    elif (x==3):
        neighborhoods_data = table_cell.text
        neighborhoods = neighborhoods_data.strip('\n')
        x=0 #reset counter
                             
        if borough == 'Not assigned':            
            borough = []
            neighborhoods_data=[]
        else:
            if neighborhoods == 'Not assigned':
                neighborhoods = borough
             
            Postcode.append(postal_code)
            Borough.append(borough)
            Neighbourhood.append(neighborhoods)

In [7]:
data=[]

for postcode_unique_element in set(Postcode):
    post_value = ''
    borough_value_var = ''
    neighbourhood_value = ''
    
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            post_value = postcode_element;
            borough_value_var = Borough[postcode_idx]
            
            if neighbourhood_value == '': 
                neighbourhood_value = Neighbourhood[postcode_idx]
            else:
                neighbourhood_value = neighbourhood_value + ', ' + Neighbourhood[postcode_idx]
    
    data.append([post_value,borough_value_var,neighbourhood_value])

df = pd.DataFrame(np.array(data),columns=['PostalCode', 'Borough', 'Neighborhood'])

In [8]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
from sklearn.cluster import KMeans

Solving environment: - 

In [ ]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)

X = df_join['Latitude']
Y = df_join['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
df_join['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(df_join['Latitude'], df_join['Longitude'], df_join['Borough'], df_join['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map)  

map